In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!pip install pyspark

The system cannot find the path specified.
'update-alternatives' is not recognized as an internal or external command,
operable program or batch file.


^C


In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.3'
spark_version = 'spark-3.5.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
hs__df = spark.read.csv(SparkFiles.get('home_sales_revised.csv'), header= True)
hs_df.show()



In [ ]:
# 2. Create a temporary view of the DataFrame.
hs_df.createOrReplaceTempView("home_sales")


In [ ]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql('''
  SELECT
    YEAR(date) as year,
    MAX(bedrooms),
    ROUND(AVG(price),2) as avg
  FROM home_sales
  WHERE bedrooms = 4
  GROUP BY year
''').show()


In [ ]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql('''
  SELECT
    date_built,
    MAX(bedrooms),
    MAX(bathrooms),
    ROUND(AVG(price),2) as avg
  FROM home_sales
  WHERE bedrooms = 3 AND bathrooms = 3
  GROUP BY date_built
''').show()


In [ ]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql('''
  SELECT
    date_built,
    MAX(bedrooms),
    MAX(bathrooms),
    MAX(floors),
    ROUND(AVG(price),2) as avg
  FROM home_sales
  WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000.00
  GROUP BY date_built
''').show()


In [ ]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()


spark.sql('''
  SELECT
    view,
    ROUND(AVG(price),2) as avg
  FROM home_sales
  GROUP BY view
  HAVING avg >= 350000
  ORDER BY view DESC
''').show()


print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 7. Cache the the temporary table home_sales.
spark.sql('cache table home_sales')

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

In [ ]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()


spark.sql('''
  SELECT
    view,
    ROUND(AVG(price), 2) as avg
  FROM home_sales
  GROUP BY view
  HAVING avg >= 350000
  ORDER BY view DESC
''').show()


print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
hs_df.write.partitionBy("date_built").parquet("p_home_sales",mode = "overwrite")

In [ ]:
# 11. Read the parquet formatted data.
phs_df = spark.read.parquet("p_home_sales")

In [ ]:
# 12. Create a temporary table for the parquet data.
ths_df.createOrReplaceTempView("temp_home_sales")

spark.sql('''
  SELECT *
  FROM temp_home_sales
  LIMIT 10
''').show()

In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()


spark.sql('''
  SELECT
    view,
    ROUND(AVG(price), 2) as avg
  FROM temp_home_sales
  GROUP BY view
  HAVING avg >= 350000
  ORDER BY view DESC
''').show()


print("--- %s seconds ---" % (time.time() - start_time))

--- 7.104873657226562e-05 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("temp_home_sales")

In [ ]:
# 15. Check if the home_sales is no longer cached
cached = spark.catalog.isCached("temp_home_sales")

# Print the result
print(f"Is temp_home_sales cached? {cached}")
